In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = 24016013 # 店舗ID
SPREADSHEET_KEY = '1cj2UbtVz9VNfRemWektP8kpWZam4rcP9FiY7R9ZiIew' # スプレッドシートのID
pstart = 1 #1 パチンコの1台目
sstart = 613 #613 スロットの1台目
wait = 0.02

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777/template_p.png"
Template_s_path = "site777/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2000":
                    s20table.append(li)
                if unit == "625":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2000":
                        s20table.append(li)
                    if unit == "625":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/19', '025559', '1', '28974.0', '-6319.0', '23', '11']
['08/19', '025851', '2', '513.0', '-902.0', '0', '0']
['08/19', '025440', '3', '7179.0', '-4212.0', '3', '2']
['08/19', '025121', '4', '4359.0', '-2707.0', '1', '1']
['08/19', '025785', '5', '7179.0', '-2406.0', '6', '3']
['08/19', '025474', '6', '1795.0', '-1203.0', '3', '1']
['08/19', '025474', '7', '1282.0', '603.0', '2', '1']
['08/19', '025747', '8', '2308.0', '603.0', '5', '2']
['08/19', '025760', '9', '0', '0', '0', '0']
['08/19', '025575', '10', '2051.0', '-1504.0', '1', '1']
['08/19', '025545', '11', '5641.0', '-1203.0', '4', '2']
['08/19', '025733', '12', '0', '0', '0', '0']
['08/19', '025776', '13', '4103.0', '-2707.0', '1', '1']
['08/19', '025776', '14', '1795.0', '-1203.0', '1', '1']
['08/19', '025885', '15', '4615.0', '-3911.0', '1', '1']
['08/19', '025885', '16', '1282.0', '3311.0', '15', '1']
['08/19', '025865', '17', '9744.0', '-5717.0', '7', '4']
['08/19', '025865', '18', '15128.0', '-2707.0', '30', '6']
['08/1

['08/19', '025665', '151', '29744.0', '-15045.0', '9', '5']
['08/19', '025665', '152', '20769.0', '-4212.0', '10', '3']
['08/19', '025665', '153', '35385.0', '-20462.0', '5', '1']
['08/19', '025665', '154', '14359.0', '-902.0', '9', '3']
['08/19', '025665', '155', '36667.0', '-26781.0', '5', '3']
['08/19', '025665', '156', '18974.0', '-9027.0', '6', '3']
['08/19', '025665', '157', '21538.0', '16853.0', '25', '6']
['08/19', '025665', '158', '22308.0', '-8726.0', '8', '3']
['08/19', '025665', '159', '29231.0', '-17453.0', '7', '4']
['08/19', '025665', '160', '0', '0', '0', '0']
['08/19', '025665', '161', '0', '0', '0', '0']
['08/19', '025665', '162', '35897.0', '18959.0', '35', '7']
['08/19', '025895', '163', '39231.0', '603.0', '25', '7']
['08/19', '025895', '164', '36667.0', '-3911.0', '21', '5']
['08/19', '025895', '165', '37949.0', '603.0', '25', '6']
['08/19', '025895', '166', '38718.0', '-6920.0', '20', '4']
['08/19', '025895', '167', '40000.0', '4515.0', '28', '6']
['08/19', '0258

['08/19', '025792', '295', '0', '0', '0', '0']
['08/19', '025792', '296', '4103.0', '-3610.0', '0', '0']
['08/19', '025792', '297', '0', '0', '0', '0']
['08/19', '025852', '298', '0', '0', '0', '0']
['08/19', '025852', '299', '2051.0', '3311.0', '6', '1']
['08/19', '025839', '300', '7949.0', '-7221.0', '0', '0']
['08/19', '025839', '301', '4103.0', '603.0', '4', '1']
['08/19', '025864', '302', '2821.0', '-2406.0', '0', '0']
['08/19', '025864', '303', '2564.0', '-2106.0', '0', '0']
['08/19', '025864', '304', '2564.0', '-2106.0', '0', '0']
['08/19', '025886', '305', '22821.0', '302.0', '15', '8']
['08/19', '025886', '306', '14359.0', '-6619.0', '5', '3']
['08/19', '025867', '307', '30769.0', '12038.0', '29', '5']
['08/19', '025867', '308', '20769.0', '-14744.0', '2', '1']
['08/19', '025867', '309', '9487.0', '-8124.0', '0', '0']
['08/19', '025893', '310', '8205.0', '-7221.0', '0', '0']
['08/19', '025849', '311', '0', '0', '0', '0']
['08/19', '025849', '312', '10256.0', '-1504.0', '5', '2

['08/19', '025521', '435', '34872.0', '-1203.0', '30', '16']
['08/19', '025521', '436', '46410.0', '5117.0', '52', '22']
['08/19', '025521', '437', '48974.0', '2709.0', '54', '22']
['08/19', '025317', '438', '17436.0', '-2406.0', '16', '7']
['08/19', '025317', '439', '8462.0', '-5115.0', '2', '1']
['08/19', '025765', '440', '39231.0', '-1203.0', '43', '16']
['08/19', '025765', '441', '44359.0', '2709.0', '54', '19']
['08/19', '025765', '442', '45385.0', '-1203.0', '50', '20']
['08/19', '025765', '443', '40256.0', '-6018.0', '37', '18']
['08/19', '025765', '444', '35385.0', '-1203.0', '39', '14']
['08/19', '025765', '445', '33846.0', '-1504.0', '37', '17']
['08/19', '025605', '446', '38974.0', '5719.0', '45', '15']
['08/19', '025605', '447', '38718.0', '4515.0', '47', '23']
['08/19', '025605', '448', '34359.0', '-3610.0', '28', '9']
['08/19', '025605', '449', '27436.0', '4214.0', '31', '15']
['08/19', '025605', '450', '41282.0', '1806.0', '42', '17']
['08/19', '025605', '451', '39231.0'

['08/19', '025165', '574', '27436.0', '3311.0', '35', '12']
['08/19', '025746', '575', '5128.0', '-3008.0', '2', '1']
['08/19', '025225', '576', '19487.0', '603.0', '22', '17']
['08/19', '025489', '577', '17179.0', '1806.0', '8', '4']
['08/19', '025471', '578', '19231.0', '603.0', '22', '5']
['08/19', '025471', '579', '17436.0', '4816.0', '23', '7']
['08/19', '025317', '580', '11795.0', '-1203.0', '16', '6']
['08/19', '025317', '581', '28205.0', '-3610.0', '30', '12']
['08/19', '025317', '582', '26667.0', '1506.0', '36', '9']
['08/19', '025317', '583', '0', '0', '0', '0']
['08/19', '025722', '584', '31026.0', '-2106.0', '37', '11']
['08/19', '025722', '585', '7692.0', '1506.0', '13', '4']
['08/19', '025722', '586', '25641.0', '-7522.0', '24', '11']
['08/19', '025668', '587', '14872.0', '9029.0', '31', '6']
['08/19', '025611', '588', '8462.0', '5117.0', '29', '3']
['08/19', '025395', '589', '9744.0', '13543.0', '24', '1']
['08/19', '025766', '590', '21026.0', '-5115.0', '18', '3']
['08/

['08/19', '120025', '711', '912', '-602.0', '1', '1', '--']
['08/19', '120025', '712', '1264', '-278.0', '3', '2', '--']
['08/19', '120025', '713', '2509', '-231.0', '8', '5', '--']
['08/19', '120025', '714', '8216', '833.0', '34', '23', '--']
['08/19', '120025', '715', '2644', '-602.0', '8', '2', '--']
['08/19', '120025', '716', '8731', '1157.0', '33', '21', '--']
['08/19', '120025', '717', '1230', '-926.0', '1', '3', '--']
['08/19', '120025', '718', '8129', '3148.0', '38', '25', '--']
['08/19', '120025', '719', '1874', '139.0', '7', '3', '--']
['08/19', '120025', '720', '2380', '-1667.0', '3', '6', '--']
['08/19', '120025', '721', '4995', '139.0', '18', '12', '--']
['08/19', '120025', '722', '7344', '2361.0', '32', '24', '--']
['08/19', '120025', '723', '1476', '-417.0', '3', '5', '--']
['08/19', '120025', '724', '4244', '139.0', '16', '9', '--']
['08/19', '120034', '725', '1396', '-972.0', '0', '1', '9']
['08/19', '120034', '726', '635', '-556.0', '0', '1', '2']
['08/19', '119824', 

['08/19', '120142', '846', '2771', '-3380.0', '0', '4', '6']
['08/19', '120142', '847', '7049', '-3611.0', '0', '14', '47']
['08/19', '120138', '848', '4012', '417.0', '0', '7', '22']
['08/19', '120138', '849', '1914', '-1389.0', '0', '2', '6']
['08/19', '120150', '850', '6762', '-2685.0', '0', '26', '63']
['08/19', '120153', '851', '4196', '185.0', '23', '0', '22']
['08/19', '120130', '852', '857', '-926.0', '0', '2', '2']
['08/19', '120130', '853', '5810', '2315.0', '0', '14', '46']
['08/19', '120130', '854', '6301', '-139.0', '0', '10', '26']
['08/19', '120080', '855', '515', '-93.0', '5', '0', '4']
['08/19', '120080', '856', '712', '-602.0', '4', '0', '1']
['08/19', '119989', '857', '1518', '-1898.0', '0', '4', '--']
['08/19', '119993', '858', '242', '-139.0', '0', '1', '1']
['08/19', '119991', '859', '374', '-324.0', '1', '0', '1']
['08/19', '119865', '860', '1436', '926.0', '0', '1', '24']
['08/19', '119860', '861', '1080', '417.0', '8', '8', '--']
['08/19', '120045', '862', '485

{'spreadsheetId': '1cj2UbtVz9VNfRemWektP8kpWZam4rcP9FiY7R9ZiIew',
 'updates': {'spreadsheetId': '1cj2UbtVz9VNfRemWektP8kpWZam4rcP9FiY7R9ZiIew',
  'updatedRange': 'S5RAW!F5840:M5879',
  'updatedRows': 40,
  'updatedColumns': 8,
  'updatedCells': 320}}